# Input

In [4]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import warnings
warnings.filterwarnings('ignore')
import xgboost
import lightgbm as lgm
import catboost

In [5]:
train = pd.read_csv('train.csv' , parse_dates= ['date_created', 'user_date_created'] )
test = pd.read_csv('test.csv' ,parse_dates= ['date_created', 'user_date_created'] )

# Pre

In [6]:
test.drop(columns = 'month_created' , inplace = True)
train = train[list(test.columns) + ['is_fake']]
train['user_agent'] = train['user_agent'].fillna('no_info')

In [8]:
startTime_user = train['user_date_created'].min()
y = train['is_fake'].astype(int)

In [9]:
from sklearn.model_selection import StratifiedKFold , cross_val_score , GridSearchCV
skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=42 )

In [6]:
train.head()

,comment,comment_id,company_id,date_created,product_id,rating,user_agent,user_date_created,user_id,user_ip,is_fake
0,компания не выполнила заказ без обьяснения пр...,1,2169053,2014-10-23 20:26:44,6.0,1,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,2010-02-03 21:03:22,91933,178.92.70.220,False
1,"Впервые встретился такой професиональный, отве...",2,709306,2016-12-10 08:38:27,2.0,5,Mozilla/5.0 (Linux; Android 5.1.1; C6602 Build...,2010-02-03 21:03:22,91933,94.179.59.162,False
2,"Несерйозная, некоректная менеджер інтернет маг...",3,35287,2017-02-11 15:55:14,NaN,1,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,2010-02-03 21:03:22,91933,37.73.137.188,False
3,"Не рекомендую. Не отвечают, не выполняют заказы.",4,1778322,2017-06-26 18:00:45,1.0,1,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,2010-02-03 21:03:22,91933,37.73.149.180,False
4,"На письма не отвечают, заказы не выполняют. Не...",5,2821533,2017-07-06 16:57:40,NaN,1,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,2010-02-03 21:03:22,91933,37.73.221.126,False


In [10]:
train['agent_encode'] = train['user_agent'].map(lambda x: str(x).lower())
for symb in [str(j) for j in range(10)] + ['_' , ':' , '-' , '[' , ']'  , '!' ,'.' , '+' ,',' , ' ']:
    print(symb)
    train['agent_encode'] = train['agent_encode'].map(lambda x : str(x).replace(symb , ''))
    
for symb in ['(' , ')' , ';' , '//' ]:
    print(symb)
    train['agent_encode'] = train['agent_encode'].map(lambda x : str(x).replace(symb , '/'))
train['agent_encode'] = train['agent_encode'].map(lambda x : str(x).replace('/' , ' '))
train['agent_encode']

0
1
2
3
4
5
6
7
8
9
_
:
-
[
]
!
.
+
,
 
(
)
;
//


0          mozilla windowsnt applewebkit khtmllikegecko c...
1          mozilla linux android cbuild a applewebkit kht...
2          mozilla linux android asusxdbuild mrak applewe...
3          mozilla linux android asusxdbuild mrak applewe...
4          mozilla linux android asusxdbuild mrak applewe...
5          mozilla windowsnt applewebkit khtmllikegecko c...
6          mozilla linux android cbuild a applewebkit kht...
7          mozilla windowsnt wow applewebkit khtmllikegec...
8                        mozilla windowsnt rv gecko firefox 
9          mozilla windowsnt applewebkit khtmllikegecko c...
10         mozilla windowsnt applewebkit khtmllikegecko c...
11         mozilla windowsnt wow applewebkit khtmllikegec...
12                   mozilla windowsnt wow rv gecko firefox 
13                   mozilla windowsnt wow rv gecko firefox 
14                   mozilla windowsnt wow rv gecko firefox 
15                   mozilla windowsnt wow rv gecko firefox 
16         mozilla windo

In [11]:
test['agent_encode'] = test['user_agent'].map(lambda x: str(x).lower())
for symb in [str(j) for j in range(10)] + ['_' , ':' , '-' , '[' , ']'  , '!' ,'.' , '+' ,',' , ' ']:
    print(symb)
    test['agent_encode'] = test['agent_encode'].map(lambda x : str(x).replace(symb , ''))
    
for symb in ['(' , ')' , ';' , '//' ]:
    print(symb)
    test['agent_encode'] = train['agent_encode'].map(lambda x : str(x).replace(symb , '/'))
test['agent_encode'] = train['agent_encode'].map(lambda x : str(x).replace('/' , ' '))
test['agent_encode']

0
1
2
3
4
5
6
7
8
9
_
:
-
[
]
!
.
+
,
 
(
)
;
//


0         mozilla windowsnt applewebkit khtmllikegecko c...
1         mozilla linux android cbuild a applewebkit kht...
2         mozilla linux android asusxdbuild mrak applewe...
3         mozilla linux android asusxdbuild mrak applewe...
4         mozilla linux android asusxdbuild mrak applewe...
5         mozilla windowsnt applewebkit khtmllikegecko c...
6         mozilla linux android cbuild a applewebkit kht...
7         mozilla windowsnt wow applewebkit khtmllikegec...
8                       mozilla windowsnt rv gecko firefox 
9         mozilla windowsnt applewebkit khtmllikegecko c...
10        mozilla windowsnt applewebkit khtmllikegecko c...
11        mozilla windowsnt wow applewebkit khtmllikegec...
12                  mozilla windowsnt wow rv gecko firefox 
13                  mozilla windowsnt wow rv gecko firefox 
14                  mozilla windowsnt wow rv gecko firefox 
15                  mozilla windowsnt wow rv gecko firefox 
16        mozilla windowsnt applewebkit 

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
train_agent_text = train['agent_encode']
test_agent_text = test['agent_encode']

all_agent_text = pd.concat([train_agent_text, test_agent_text])

vectorizer_agent = TfidfVectorizer( min_df = 10 ,lowercase= True , ngram_range = (1 , 5))
vectorizer_agent.fit(all_agent_text)

train_agent = vectorizer_agent.transform(train_agent_text)
test_agent = vectorizer_agent.transform(test_agent_text)

vectorizer_agent

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(1, 5), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
train_comment_text = train['comment']
test_comment_text = test['comment']

all_comment_text = pd.concat([train_comment_text, test_comment_text])

vectorizer_comment = TfidfVectorizer( min_df = 10 ,lowercase= True , ngram_range = (1 , 1))
vectorizer_comment.fit(all_comment_text)

train_comment1 = vectorizer_comment.transform(train_comment_text)
test_comment1 = vectorizer_comment.transform(test_comment_text)

vectorizer_comment = TfidfVectorizer( min_df = 10 ,lowercase= True , ngram_range = (1 , 2))
vectorizer_comment.fit(all_comment_text)

train_comment2 = vectorizer_comment.transform(train_comment_text)
test_comment2 = vectorizer_comment.transform(test_comment_text)

vectorizer_comment

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [14]:
train_agent

<1281499x23483 sparse matrix of type '<class 'numpy.float64'>'
	with 31055489 stored elements in Compressed Sparse Row format>

In [16]:
train_comment2

<1281499x295321 sparse matrix of type '<class 'numpy.float64'>'
	with 37540673 stored elements in Compressed Sparse Row format>

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
classifier = LogisticRegression(C =10 , solver ='sag' , n_jobs =-1) 

In [18]:
from sklearn.model_selection import cross_val_predict
import scipy
train_agent_and_comment =  pd.DataFrame(columns = ['log_comment' , 'log_agent' , 'log_coment_agent'] , index = train.index)
train_agent_and_comment['log_agent'] =cross_val_predict(classifier , train_agent , y , n_jobs= -1 , cv = skf , verbose =2, method='predict_proba')[:,1]
train_agent_and_comment['log_comment'] =cross_val_predict(classifier , train_comment2 , y , n_jobs= -1 , cv = skf, verbose =2, method='predict_proba')[:,1]
train_agent_and_comment['log_coment_agent'] =cross_val_predict(classifier , 
                                                               scipy.sparse.hstack([train_agent , train_comment1]) ,
                                                               y , n_jobs= -1 , cv = skf, verbose =2, method='predict_proba')[:,1]


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  2.0min remaining:  3.0min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.0min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.0min finished
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.7min remaining:  2.6min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.9min finished


NameError: name 'scipy' is not defined

In [31]:
train_agent_and_comment.head()

,log_comment,log_agent,log_coment_agent
0,0.000664,0.000603,6.094652e-05
1,0.001020,0.000498,4.613379e-05
2,0.000657,0.000064,2.298313e-06
3,0.000301,0.000081,8.855684e-07
4,0.000100,0.000078,3.351581e-06


In [35]:
from sklearn.metrics import log_loss
for col in train_agent_and_comment.columns:
    print(log_loss(y ,train_agent_and_comment[col]))

0.01222373636880763
0.017384898299485998
0.012519850646921833


In [47]:
test_agent_and_comment =  pd.DataFrame(columns = ['log_comment' , 'log_agent' , 'log_comment_agent'] , index = test.index)
classifier.fit( train_agent , y)
test_agent_and_comment['log_agent'] = classifier.predict_proba( test_agent)[:,1]
print(1)
classifier.fit( train_comment2 , y)
test_agent_and_comment['log_comment'] = classifier.predict_proba( test_comment2)[:,1]
print(2)
classifier.fit( scipy.sparse.hstack([train_agent , train_comment1]) , y)
test_agent_and_comment['log_comment_agent'] = classifier.predict_proba( scipy.sparse.hstack([test_agent , test_comment1]))[:,1]

1
2


In [48]:
test_agent_and_comment.sum()

log_comment          767.825714
log_agent            497.338502
log_coment_agent       0.000000
log_comment_agent    381.892231
dtype: float64

In [57]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
def __encode_categorical(df_list, cat_cols):
    d = defaultdict(LabelEncoder)
    codes = pd.concat(
        [df[cat_cols] for df in df_list],
        axis=0
    ).fillna('').apply(
        lambda x: d[x.name].fit(x)
    ),
    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(
            lambda x: d[x.name].transform(x))

__encode_categorical(df_list=[test,train], cat_cols=['agent_encode'])

In [58]:
test_agent_and_comment['agent_encode'] = test['agent_encode']
train_agent_and_comment['agent_encode'] = train['agent_encode']

In [59]:
train_agent_and_comment.head()

,log_comment,log_agent,log_coment_agent,agent_encode
0,0.000664,0.000603,6.094652e-05,12049
1,0.001020,0.000498,4.613379e-05,2437
2,0.000657,0.000064,2.298313e-06,2103
3,0.000301,0.000081,8.855684e-07,2103
4,0.000100,0.000078,3.351581e-06,2103


In [60]:
test_agent_and_comment.head()

,log_comment,log_agent,log_comment_agent,agent_encode
0,0.001709,0.000597,0.022145,12049
1,0.000019,0.000408,0.000002,2437
2,0.002556,0.000069,0.000027,2103
3,0.000755,0.000069,0.000055,2103
4,0.009095,0.000069,0.000030,2103


In [61]:
train_agent_and_comment.to_csv('train_agent_and_comment.csv')
test_agent_and_comment.to_csv('test_agent_and_comment.csv')